In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [2]:
bs,bptt = 64,70

## Language modeling

### Data

The [large movie view dataset](http://ai.stanford.edu/~amaas/data/sentiment/) contains a collection of 50,000 reviews from IMDB. The dataset contains an even number of positive and negative reviews. The authors considered only highly polarized reviews. A negative review has a score ≤ 4 out of 10, and a positive review has a score ≥ 7 out of 10. Neutral reviews are not included in the dataset. The dataset is divided into training and test sets. The training set is the same 25,000 labeled reviews.

The **sentiment classification task** consists of predicting the polarity (positive or negative) of a given text.

However, before we try to classify *sentiment*, we will simply try to create a *language model*; that is, a model that can predict the next word in a sentence. Why? Because our model first needs to understand the structure of English, before we can expect it to recognize positive vs negative sentiment.

So our plan of attack is the same as we used for Dogs v Cats: pretrain a model to do one thing (predict the next word), and fine tune it to do something else (classify sentiment).

Unfortunately, there are no good pretrained language models available to download, so we need to create our own.

In [2]:
PATH='data/aclImdb/'

TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

aclImdb_v1.tar.gz  imdbEr.txt  imdb.vocab  models/  README  test/  tmp/  train/


Let's look inside the training folder...

In [8]:
trn_files = !ls {TRN}
trn_files[:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '1000_0.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt']

...and at an example review.

In [20]:
review = !cat {TRN}{trn_files[6]}
review[0]

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop-socky fung-ku, but what I got instead was a comedy. So, it wasn't quite was I was expecting, but I really liked it anyway! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them!! I was laughing my ass off. I mean, the cops were just so bad! And when I say bad, I mean The Shield Vic Macky bad. But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose...man, oh man. What can you say about that hottie. She was great and put those other actresses to shame. She should work more often!!!!! I also really liked the fight scene outside of the building. That was done really well. Lots of fighting and people getting their heads banged up. FUN! Last, but not least Joe Estevez and William Smith were great as the...well, I wasn't sure what they were, but they seemed to be having fun and throwing out 

Sounds like I'd really enjoy *Zombiegeddon*...

Now we'll check how many words are in the dataset.

In [5]:
!find {TRN} -name '*.txt' | xargs cat | wc -w

17486581


In [6]:
!find {VAL} -name '*.txt' | xargs cat | wc -w

5686719


Before we can analyze text, we must first *tokenize* it. This refers to the process of splitting a sentence into an array of words (or more generally, into an array of *tokens*).

In [22]:
' '.join(spacy_tok(review[0]))

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop - socky fung - ku , but what I got instead was a comedy . So , it was n't quite was I was expecting , but I really liked it anyway ! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them ! ! I was laughing my ass off . I mean , the cops were just so bad ! And when I say bad , I mean The Shield Vic Macky bad . But unlike that show I was laughing when they shot people and smoked dope . \n\n Felissa Rose ... man , oh man . What can you say about that hottie . She was great and put those other actresses to shame . She should work more often ! ! ! ! ! I also really liked the fight scene outside of the building . That was done really well . Lots of fighting and people getting their heads banged up . FUN ! Last , but not least Joe Estevez and William Smith were great as the ... well , I was n't sure what they were , but they seemed t

We use Pytorch's [torchtext](https://github.com/pytorch/text) library to preprocess our data, telling it to use the wonderful [spacy](https://spacy.io/) library to handle tokenization.

First, we create a torchtext *field*, which describes how to preprocess a piece of text - in this case, we tell torchtext to make everything lowercase, and tokenize it with spacy.

In [23]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

fastai works closely with torchtext. We create a ModelData object for language modeling by taking advantage of `LanguageModelData`, passing it our torchtext field object, and the paths to our training, test, and validation sets. In this case, we don't have a separate test set, so we'll just use `VAL_PATH` for that too.

As well as the usual `bs` (batch size) parameter, we also not have `bptt`; this define how many words are processing at a time in each row of the mini-batch. More importantly, it defines how many 'layers' we will backprop through. Making this number higher will increase time and memory requirements, but will improve the model's ability to handle long sentences.

In [25]:
bs=64; bptt=80

In [26]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

After building our `ModelData` object, it automatically fills the `TEXT` object with a very important attribute: `TEXT.vocab`. This is a *vocabulary*, which stores which words (or *tokens*) have been seen in the text, and how each word will be mapped to a unique integer id. We'll need to use this information again later, so we save it.

*(Technical note: python's standard `Pickle` library can't handle this correctly, so at the top of this notebook we used the `dill` library instead and imported it as `pickle`)*.

In [27]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

Here are the: # batches; # unique tokens in the vocab; # tokens in the training set; # sentences

In [9]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(4602, 34945, 1, 20621966, 34945)

This is the start of the mapping from integer IDs to unique tokens.

In [31]:
# 'itos': 'int-to-string'
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'it', 'in']

In [35]:
# 'stoi': 'string to int'
TEXT.vocab.stoi['the']

2

Note that in a `LanguageModelData` object there is only one item in each dataset: all the words of the text joined together.

In [37]:
md.trn_ds[0].text[:12]

['i',
 'just',
 'do',
 "n't",
 'get',
 'some',
 'of',
 'the',
 'big',
 'premises',
 'of',
 'this']

torchtext will handle turning this words into integer IDs for us automatically.

In [40]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

Variable containing:
   12
   51
   53
   30
   94
   61
    7
    2
  214
 9427
    7
   13
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

Our `LanguageModelData` object will create batches with 64 columns (that's our batch size), and varying sequence lengths of around 80 tokens (that's our `bptt` parameter - *backprop through time*).

Each batch also contains the exact same data as labels, but one word later in the text - since we're trying to always predict the next word. The labels are flattened into a 1d array.

In [41]:
next(iter(md.trn_dl))

(Variable containing:
     12    687     13  ...    1923     19     17
     51    615     25  ...      10     63     51
     53      3      3  ...      42    428    833
         ...            ⋱           ...         
     97     36     17  ...       2   3646     23
     53    307  25441  ...     997   9466      4
     30      3   3916  ...     150     46     20
 [torch.cuda.LongTensor of size 77x64 (GPU 0)], Variable containing:
     51
    615
     25
   ⋮   
     13
     50
    659
 [torch.cuda.LongTensor of size 4928 (GPU 0)])

### Train

We have a number of parameters to set - we'll learn more about these later, but you should find these values suitable for many problems.

In [43]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

Researchers have found that large amounts of *momentum* (which we'll learn about later) don't work well with these kinds of *RNN* models, so we create a version of the *Adam* optimizer with less momentum than it's default of `0.9`.

In [44]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

fastai uses a variant of the state of the art [AWD LSTM Language Model](https://arxiv.org/abs/1708.02182) developed by Stephen Merity. A key feature of this model is that it provides excellent regularization through [Dropout](https://en.wikipedia.org/wiki/Convolutional_neural_network#Dropout). There is no simple way known (yet!) to find the best values of the dropout parameters below - you just have to experiment...

However, the other parameters (`alpha`, `beta`, and `clip`) shouldn't generally need tuning.

In [45]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

As you can see below, I gradually tuned the language model in a few stages. I possibly could have trained it further (it wasn't yet overfitting), but I didn't have time to experiment more. Maybe you can see if you can train it to a better accuracy! (I used `lr_find` to find a good learning rate, but didn't save the output in this notebook. Feel free to try running it yourself now.)

In [11]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.      4.6754  4.5553]                                     
[ 1.      4.5344  4.4255]                                     
[ 2.      4.4838  4.389 ]                                     
[ 3.      4.5597  4.4271]                                     
[ 4.      4.4565  4.3472]                                     
[ 5.      4.4095  4.3167]                                     
[ 6.      4.5065  4.3852]                                     
[ 7.      4.4273  4.3122]                                     
[ 8.      4.3651  4.2831]                                     



In [12]:
learner.save_encoder('adam1_enc')

In [13]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=10, cycle_save_name='adam3_10')

A Jupyter Widget

[ 0.      4.5019  4.3872]                                     
[ 1.      4.4707  4.3654]                                     
[ 2.      4.4606  4.3432]                                     
[ 3.      4.4183  4.3185]                                     
[ 4.      4.3858  4.2928]                                     
[ 5.      4.368   4.2717]                                     
[ 6.      4.3286  4.2485]                                     
[ 7.      4.3101  4.2333]                                     
[ 8.      4.2913  4.2256]                                     
[ 9.      4.2581  4.2244]                                     
[ 10.       4.4615   4.3373]                                  
[ 11.       4.4281   4.3265]                                  
[ 12.       4.4209   4.3076]                                  
[ 13.       4.3941   4.2883]                                  
[ 14.       4.3655   4.2645]                                  
[ 15.       4.3298   4.242 ]                           

In [14]:
learner.save_encoder('adam3_10_enc')

In [15]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20, cycle_save_name='adam3_20')

A Jupyter Widget

[ 0.      4.2122  4.1803]                                     
[ 1.      4.4048  4.2924]                                     
[ 2.      4.4049  4.292 ]                                     
[ 3.      4.3769  4.2839]                                     
[ 4.      4.3734  4.2757]                                     
[ 5.      4.3558  4.2684]                                     
[ 6.      4.3471  4.2587]                                     
[ 7.      4.3372  4.2475]                                     
[ 8.      4.3426  4.2398]                                     
[ 9.      4.3155  4.2253]                                     
[ 10.       4.2975   4.2165]                                  
[ 11.       4.2747   4.2057]                                  
[ 12.       4.2725   4.1949]                                  
[ 13.       4.2533   4.1861]                                  
[ 14.       4.2281   4.1788]                                  
[ 15.       4.226    4.1729]                           

In [47]:
learner.load_cycle('adam3_20', 0)

In the sentiment analysis section, we'll just need half of the language model - the *encoder*, so we save that part.

In [16]:
learner.save_encoder('adam3_20_enc')

Language modeling accuracy is generally measured using the metric *perplexity*, which is simply `exp()` of the loss function we used.

In [17]:
math.exp(4.165)

64.3926824434624

### Test

We can play around with our language model a bit to check it seems to be working OK. First, let's create a short bit of text to 'prime' a set of predictions. We'll use our torchtext field to numericalize it so we can feed it to our language model.

In [62]:
m=learner.model
ss=""". So, it wasn't quite was I was expecting, but I really liked it anyway! The best"""
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

". So , it was n't quite was I was expecting , but I really liked it anyway ! The best"

We haven't yet added methods to make it easy to test a language model, so we'll need to manually go through the steps.

In [68]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

Let's see what the top 10 predictions were for the next word after our short text:

In [20]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['performance',
 'of',
 'friend',
 'actor',
 'thing',
 'scene',
 'character',
 'part',
 'line',
 'movie']

...and let's see if our model can generate a bit more text all by itself!

In [69]:
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

. So, it wasn't quite was I was expecting, but I really liked it anyway! The best 

performance was the one in the movie where he was a little too old for the part . i think he was a good actor , but he was n't that good . 

 the movie was a bit slow , but it was n't too bad . the acting ...


### Sentiment

We'll need to the saved vocab from the language model, since we need to ensure the same words map to the same IDs.

In [70]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

`sequential=False` tells torchtext that a text field should be tokenized (in this case, we just want to store the 'positive' or 'negative' single label).

`splits` is a torchtext method that creates train, test, and validation sets. The IMDB dataset is built into torchtext, so we can take advantage of that. Take a look at `lang_model-arxiv.ipynb` to see how to define your own fastai/torchtext datasets.

In [71]:
IMDB_LABEL = data.Field(sequential=False)
splits = torchtext.datasets.IMDB.splits(TEXT, IMDB_LABEL, 'data/')

fastai can create a ModelData object directly from torchtext splits.

In [72]:
md2 = TextData.from_splits(PATH, splits, bs)

In [73]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'adam3_20_enc')

Because we're fine-tuning a pretrained model, we'll use differential learning rates, and also increase the max gradient for clipping, to allow the SGDR to work better.

In [74]:
m3.clip=25.
lrs=np.array([1e-4,1e-3,1e-2])

In [75]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

A Jupyter Widget

[ 0.       0.43908  0.3433   0.84732]                        



A Jupyter Widget

[ 0.       0.30647  0.19581  0.92805]                        



In [59]:
m3.fit(lrs, 8, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

A Jupyter Widget

[ 0.      0.275   0.1847  0.9306]                            
[ 1.      0.2528  0.1781  0.9336]                            
[ 2.      0.2448  0.1729  0.9365]                            
[ 3.      0.2052  0.1721  0.9378]                            
[ 4.      0.2239  0.1619  0.9413]                            
[ 5.      0.1956  0.1628  0.9417]                            
[ 6.      0.2093  0.158   0.9421]                            
[ 7.      0.1733  0.1591  0.9432]                            
[ 8.      0.1916  0.1552  0.9443]                            
[ 9.      0.1652  0.157   0.9451]                            
[ 10.       0.1749   0.1609   0.9427]                        
[ 11.       0.147    0.1579   0.9441]                        
[ 12.       0.1554   0.1626   0.942 ]                        
[ 13.       0.1302   0.1603   0.9437]                        
[ 14.       0.1346   0.1802   0.9384]                        
[ 15.       0.1425   0.1711   0.941 ]                        



In [78]:
m3.load_cycle('imdb2', 4)

In [79]:
accuracy(*m3.predict_with_targs())

0.94511217948717952

In [ ]:
def pred_snapshot(learner, name, cycle):
    learner.load_cycle(name, cycle)
    return learner.predict_with_targs()

cyc_preds = [pred_snapshot(m3, 'imdb2', i) for i in range(4,7)]

In [15]:
preds = [p[0] for p in cyc_preds[:]]
y = cyc_preds[0][1]
print(accuracy(np.mean(preds,0),y))

### End